In [1]:
import numpy as np
import argopy
from argopy import DataFetcher
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import gsw
from datetime import datetime
from cartopy.geodesic import Geodesic
from scipy import linalg
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ConstantKernel as C
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import haversine_distances
from matplotlib.patches import Circle

import ArgoHeatContentDataCollater as ahdc
import ArgoGPR as ag

In [2]:
# 1. LOAD DATA
# Adjust path to your folder
df_argo = ahdc.load_argo_data_advanced(
    nc_dir='/home/avik2007/ArgoEBUSAnalysis/ArgoData/', 
    start_date='2018-01-01', 
    end_date='2018-03-01', # Grab a few months to test
    lat_bounds=[30, 40], 
    lon_bounds=[-126, -119]
)


📂 FOUND LOCAL DATASET: /home/avik2007/ArgoEBUSAnalysis/ArgoData/argo_2018-01-01_to_2018-03-01_lat30_40_lon-126_-119_z0_200.pkl
   Loading processed DataFrame...
   ✅ Loaded 2127 observations from disk.


In [3]:
jan_mask = [(df_argo['date'][i].month == 1) for i in range(0, len(df_argo))]
df_argo_january = df_argo[jan_mask]

In [4]:
# 2. RUN LOOO (Baseline Check)
# This validates if the model works mathematically
z_looo = ag.generalized_cross_validation(
    df_argo_january, 
    feature_cols=['lat', 'lon'], # 2D Spatial Only for now
    method='KFold', k_fold_data_percent=10
)


🚀 STARTING GLOBAL VALIDATION: KFold
   🤖 AutoTuning: Running 5 iterations on 100 points (5.0%) to estimate correlation lengths/times...
      (Constraint: Length Scale capped at 7.12 standard deviations)


/home/avik2007/miniconda3/envs/oceanography/lib/python3.14/site-packages/sklearn/gaussian_process/kernels.py:450: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k2__length_scale is close to the specified upper bound 7.115248613095652. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/avik2007/miniconda3/envs/oceanography/lib/python3.14/site-packages/sklearn/model_selection/_split.py:87: UserWarning: The groups parameter is ignored by KFold
  warnings.warn(


      ✅ LEARNED HYPERPARAMETERS (Avg of 5 runs):
         Noise (Uncertainty): ±2.358 °C
         Correlation Lengths:
           - lat: 5.306°  (~589 km)
           - lon: 0.488°  (~44 km at 35.5N)
   ⚡ Strategy: 10-Fold CV (Testing 10% per fold)

✅ RESULTS (KFold):
   RMSE:                2.384 °C
   Rel. Error (RMSRE):  0.2107 (dimensionless)
   Mean Z:              -0.002
   Std Z:               1.007 (Ideal: 1.0)


In [5]:


# 3. RUN LOFO (The "Hard" Test)
# This validates if the model works SCIENTIFICALLY
z_lofo = ag.generalized_cross_validation(
    df_argo_january, 
    feature_cols=['lat', 'lon'], 
    method='LOFO'
)


🚀 STARTING GLOBAL VALIDATION: LOFO
   🤖 AutoTuning: Running 5 iterations on 100 points (5.0%) to estimate correlation lengths/times...
      (Constraint: Length Scale capped at 7.12 standard deviations)


/home/avik2007/miniconda3/envs/oceanography/lib/python3.14/site-packages/sklearn/gaussian_process/kernels.py:450: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k2__length_scale is close to the specified upper bound 7.115248613095652. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


      ✅ LEARNED HYPERPARAMETERS (Avg of 5 runs):
         Noise (Uncertainty): ±2.305 °C
         Correlation Lengths:
           - lat: 4.372°  (~485 km)
           - lon: 0.406°  (~37 km at 35.5N)


ValueError: The 'groups' parameter should not be None.